# Task 1

## Install Dependencies

In [75]:
!pip install python-dotenv langchain_google_genai langchain langchain-community pypdf fpdf

## Load API key and Initial LLM

In [76]:
# Run this cell and paste the API key in the prompt
import os
import getpass

GOOGLE_API_KEY = getpass.getpass('Gemini API Key:')

Gemini API Key:··········


In [77]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.7, top_p=0.85, google_api_key=GOOGLE_API_KEY)

## Load an entire book

In [78]:
from langchain.document_loaders import PyPDFLoader

# Load the book
loader = PyPDFLoader("/content/crime-and-punishment.pdf")
pages = loader.load()

# Combine the pages, and replace the tabs with spaces
text = ""

for page in pages:
    text += page.page_content

text = text.replace('\t', ' ')

In [79]:
num_tokens = llm.get_num_tokens(text)

print (f"This book has {num_tokens} tokens in it")

This book has 314292 tokens in it


## Creating Embeddings

In [80]:
# Loaders
from langchain.schema import Document

# Splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Model
from langchain.chat_models import ChatOpenAI

# Embedding Support
from langchain.vectorstores import FAISS
# from langchain.embeddings import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Prompts
from langchain import PromptTemplate

# Summarizer we'll use for Map Reduce
from langchain.chains.summarize import load_summarize_chain

# Data Science
import numpy as np
from sklearn.cluster import KMeans

In [81]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=10000, chunk_overlap=3000)

docs = text_splitter.create_documents([text])

In [82]:
num_documents = len(docs)

print (f"Now our book is split up into {num_documents} documents")

Now our book is split up into 167 documents


In [83]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)

vectors = embeddings.embed_documents([x.page_content for x in docs])

Now let's cluster our embeddings.

## Creating Clusters

In [84]:
num_clusters = 11

# Perform K-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(vectors)

In [85]:
kmeans.labels_

array([ 9,  9,  9,  1,  0,  0,  0,  0,  0,  3,  6,  6,  6,  6,  6,  3,  1,
        1,  4,  0,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  5,  5,  1,  1,  1,
        1,  1,  1,  1,  0,  0,  0,  0,  3,  1,  6,  6,  6,  1,  1,  6,  6,
        6,  6,  4,  6,  6,  3,  6,  3,  8,  8,  8,  2,  2,  2,  2,  8,  1,
        9,  4,  4,  4,  6,  6,  6,  6,  6,  6,  6,  3,  3,  3,  7,  7,  8,
        8,  2,  8,  8,  8,  8,  8,  8,  5,  5,  5,  5,  0,  0,  0,  0,  0,
        5,  3,  5,  5,  3,  7,  7,  7,  7,  3,  3,  0,  0,  0,  3,  3,  8,
        8,  8,  8,  8,  8, 10,  4,  4,  4,  4, 10, 10, 10,  6, 10, 10, 10,
       10, 10, 10,  6,  6,  6,  3,  1,  1,  1,  6,  6,  3,  3],
      dtype=int32)

In [86]:
# Find the closest embeddings to the centroids

# Create an empty list that will hold your closest points
closest_indices = []

# Loop through the number of clusters you have
for i in range(num_clusters):

    # Get the list of distances from that particular cluster center
    distances = np.linalg.norm(vectors - kmeans.cluster_centers_[i], axis=1)

    # Find the list position of the closest one (using argmin to find the smallest distance)
    closest_index = np.argmin(distances)

    # Append that position to your closest indices list
    closest_indices.append(closest_index)

Now sort them (so the chunks are processed in order)

In [87]:
selected_indices = sorted(closest_indices)
selected_indices

[24, 58, 60, 82, 92, 96, 105, 122, 125, 143, 148]

## Creating summaries of each cluster

In [88]:
llm3 = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.7, top_p=0.85, google_api_key=GOOGLE_API_KEY)

In [89]:
map_prompt = """
You will be given a single passage of a book. This section will be enclosed in triple backticks (```)
Your goal is to give a summary of this section so that a reader will have a full understanding of what happened.
Your response should be at least three paragraphs and fully encompass what was said in the passage.

```{text}```
FULL SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

In [90]:
map_chain = load_summarize_chain(llm=llm3,
                             chain_type="stuff",
                             prompt=map_prompt_template)

Then go get your docs which the top vectors represented.

In [91]:
selected_docs = [docs[doc] for doc in selected_indices]

Let's loop through our selected docs and get a good summary for each chunk. We'll store the summary in a list.

In [92]:
# Make an empty list to hold your summaries
summary_list = []

# Loop through a range of the lenght of your selected docs
for i, doc in enumerate(selected_docs):

    # Go get a summary of the chunk
    chunk_summary = map_chain.run([doc])

    # Append that summary to your list
    summary_list.append(chunk_summary)

    print (f"Summary #{i} (chunk #{selected_indices[i]}) - Preview: {chunk_summary[:250]} \n")

Summary #0 (chunk #24) - Preview: Raskolnikov's plan to murder the pawnbroker was disrupted when he found her unexpectedly home. He had not anticipated this and was overwhelmed with frustration and humiliation. However, he noticed an axe in the porter's room and seized it as a weapon 

Summary #1 (chunk #58) - Preview: Katerina Ivanovna arrives at the scene of her husband's accident, where a priest is administering last rites. She angrily dismisses the priest's words of comfort and reveals her resentment towards her husband, accusing him of bringing misery and pove 

Summary #2 (chunk #60) - Preview: Raskolnikov returns home with Razumihin, who reveals that Porfiry and Ilya Petrovitch suspect Raskolnikov of madness. Raskolnikov is agitated and denies being mad. He tells Razumihin about visiting a dying clerk and encountering someone with a flame- 

Summary #3 (chunk #82) - Preview: Raskolnikov is interrogated by Porfiry Petrovitch, a police investigator, about the murder of Alyona Ivan

## Combined Summary

In [93]:
summaries = "\n".join(summary_list)

# Convert it back to a document
summaries = Document(page_content=summaries)

print (f"Your total summary has {llm.get_num_tokens(summaries.page_content)} tokens")

Your total summary has 2551 tokens


In [94]:
llm4 = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.7, top_p=0.85, google_api_key=GOOGLE_API_KEY)

In [95]:
combine_prompt = """
You will be given a series of summaries from a book. The summaries will be enclosed in triple backticks (```)
Your goal is to give a verbose summary of what happened in the story.
The reader should be able to grasp what happened in the book.

```{text}```
VERBOSE SUMMARY:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

In [96]:
reduce_chain = load_summarize_chain(llm=llm4,
                             chain_type="stuff",
                             prompt=combine_prompt_template,
#                              verbose=True # Set this to true if you want to see the inner workings
                                   )

Run! Note this will take a while

In [97]:
output = reduce_chain.run([summaries])

In [98]:
print(output)

In the depths of his poverty and desperation, Raskolnikov hatched a plan to murder the pawnbroker, Alyona Ivanovna, in order to steal her money and escape his financial woes. However, his plan was disrupted when he found her unexpectedly home. Overwhelmed by frustration and humiliation, he seized an axe from the porter's room and proceeded towards her apartment.

As he approached the building, Raskolnikov's mind wandered to irrelevant thoughts, a distraction from the gravity of his mission. Upon reaching the pawnbroker's door, he listened intently and heard a faint noise. Suspecting someone was listening from within, he rang the bell several times. After a moment, the latch was unfastened, but the door was opened only slightly, revealing the pawnbroker's suspicious eyes. Raskolnikov made the mistake of pulling the door open, nearly dragging the old woman out with it. She retreated in alarm as Raskolnikov entered the apartment uninvited.

Inside, Raskolnikov found the pawnbroker's half-

## Create PDF

In [99]:
from fpdf import FPDF

def create_pdf(content, output_path="output.pdf"):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    for item in content:
        pdf.multi_cell(0, 10, item)
        pdf.ln()  # Add a line break between items

    pdf.output(output_path)

# Execute the chain with delay and create a PDF
try:
    text = output.split('\n\n')
    # processed_results = process_with_delay(docs)
    create_pdf(text, "Crime_and_Punishment_Summary.pdf")
    print("PDF created successfully.")
except Exception as e:
    print(f"Error: {e}")

PDF created successfully.


# Task 2

In [100]:
from langchain.prompts import PromptTemplate

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["name", "field_of_study", "year_of_study", "subjects", "learning_styles", "objectives", "challenges", "extracurricular_activities"],
    template="""
    ### Instructions
    You are an AI academic advisor tasked with creating personalized study plans for students. Each student has a unique profile, and your goal is to design a study plan that not only addresses their academic requirements but also aligns with their personal preferences, goals, and challenges.

    ### Student Information
    - Name: {name}
    - Field of Study: {field_of_study}
    - Year of Study: {year_of_study}
    - Subjects: {subjects}
    - Preferred Learning Styles: {learning_styles}
    - Personal Objectives: {objectives}
    - Challenges: {challenges}
    - Extracurricular Activities: {extracurricular_activities}

    ### Desired Study Plan Format
    Based on the above information, create a comprehensive study plan that includes the following components:

    1. Subject-wise Study Plan:
        - Provide a detailed study schedule for each subject.
        - Include recommended resources (books, online courses, videos) tailored to the student's learning style.
        - Suggest specific study techniques that align with the student's preferred learning styles (visual, auditory, kinesthetic).
        - Example:
          - Subject: Mathematics
          - Schedule: Study for 1 hour daily in the morning
          - Resources: "Khan Academy" videos, "Calculus" by James Stewart
          - Techniques: Use visual aids like graphs and diagrams

    2. Personal Objectives and Challenges:
        - Address the student's personal objectives, such as preparing for a specific exam or achieving a particular grade.
        - Provide strategies to overcome any challenges or learning difficulties mentioned.
        - Example:
          - Objective: Prepare for final exams
          - Strategy: Create a revision timetable starting 2 months before exams
          - Challenge: Difficulty with advanced calculus
          - Solution: Weekly tutoring sessions, practice problems

    3. Integration of Extracurricular Activities:
        - Suggest a balanced schedule that allows the student to manage their academic workload alongside their extracurricular activities.
        - Recommend ways to integrate learning opportunities from extracurricular activities into the study plan.
        - Example:
          - Activity: Robotics club
          - Integration: Apply physics and programming concepts from coursework to robotics projects

    4. Regular Assessments and Adjustments:
        - Propose a system for regular self-assessment to monitor progress.
        - Include checkpoints for reviewing and adjusting the study plan based on performance and feedback.
        - Example:
          - Assessment: Bi-weekly quizzes for each subject
          - Checkpoints: Monthly review meetings with a mentor

    5. Motivation and Well-being:
        - Incorporate motivational strategies to keep the student engaged and motivated.
        - Suggest practices for maintaining well-being and managing stress.
        - Example:
          - Motivation: Set short-term goals and reward achievements
          - Well-being: Practice mindfulness and take regular breaks

    Ensure that the study plan is detailed, actionable, and personalized to meet the student's unique needs and aspirations.
    """
)

# Example usage
input_data = {
    "name": "Ammar Hussain",
    "field_of_study": "Mechatronics Engineering",
    "year_of_study": "3rd Year",
    "subjects": "Mathematics, Physics, Computer Science, Electronics",
    "learning_styles": "Visual, Kinesthetic",
    "objectives": "Prepare for final exams, master machine learning concepts",
    "challenges": "Difficulty with advanced calculus, managing time effectively",
    "extracurricular_activities": "Robotics club, Football team"
}

# Generate the prompt
prompt = prompt_template.format(**input_data)
print(prompt)



    ### Instructions
    You are an AI academic advisor tasked with creating personalized study plans for students. Each student has a unique profile, and your goal is to design a study plan that not only addresses their academic requirements but also aligns with their personal preferences, goals, and challenges. 

    ### Student Information
    - Name: Ammar Hussain
    - Field of Study: Mechatronics Engineering
    - Year of Study: 3rd Year
    - Subjects: Mathematics, Physics, Computer Science, Electronics
    - Preferred Learning Styles: Visual, Kinesthetic
    - Personal Objectives: Prepare for final exams, master machine learning concepts
    - Challenges: Difficulty with advanced calculus, managing time effectively
    - Extracurricular Activities: Robotics club, Football team

    ### Desired Study Plan Format
    Based on the above information, create a comprehensive study plan that includes the following components:

    1. Subject-wise Study Plan:
        - Provide a detai